In [1]:
import math
import random
import re
import timeit
from collections import namedtuple
from random import Random

import pandas as pd

import NFD_SA
from algorithm_wrappers import concat_placement
from c_interop import LIBRARY_FILE, Library
from common import Item
from data_analys.data_generator import guillotine_cutting, guillotine_cutting_max
from ortools_models import task_one_model, task_two_model
from problem_io import load_instance, save_instance
from skyline import skyline_decode

In [2]:
# noinspection PyTypeChecker
Instance = namedtuple('Instance', ['pallet_width', 'pallet_height', 'items'])

In [48]:
gen_types = ['random', 'guillotine_rand', 'guillotine_equal', 'guillotine_max']


def generate_instance(w: int, h: int, items_count: int, gen_type: str, seed: int = 1, min_side: int = 5,
                      max_side: int = 20) -> Instance:
    if gen_type == 'random':
        rng = Random(seed)
        return Instance(w, h, [Item(rng.randint(min_side, max_side), rng.randint(min_side, max_side)) for _ in
                               range(items_count)])
    if gen_type.startswith('guillotine'):
        if gen_type == 'guillotine_rand':
            items_xywh = guillotine_cutting(w, h, items_count, min_side, '', seed)
        elif gen_type == 'guillotine_equal':
            items_xywh = guillotine_cutting(w, h, items_count, min_side, seed=seed)
        elif gen_type == 'guillotine_max':
            items_xywh = guillotine_cutting_max(w, h, items_count, min_side, seed)
        else:
            assert False

        items_wh = [(w, h) if w > h else (h, w)
                    for x, y, w, h in items_xywh
                    ]
        items_cl = [Item(int(w), int(h)) for w, h in items_wh]
        return Instance(w, h, items_cl)


def generate_random(items_count: int, seed: int = 1, min_side: int = 5, max_side: int = 20) -> Instance:
    rng = Random(seed)
    items_cl = [Item(rng.randint(min_side, max_side), rng.randint(min_side, max_side)) for _ in range(items_count)]
    area = sum(it.width * it.height for it in items_cl)
    w = round(math.sqrt(area))
    h = round(math.sqrt(area))
    return Instance(w, h, items_cl)

In [5]:
random_instances = [(gen_type, seed, generate_instance(100, 100, n, gen_type, seed)) for gen_type in gen_types for n in
                    [10, 20, 50, 100] for seed in range(1)]

In [51]:
add_random_instances = [('random', seed, generate_random(n, seed)) for n in [10, 20, 50, 100] for seed in [1]]

In [53]:
for gen_type, seed, instance in add_random_instances:
    save_instance(f'data/random/{gen_type}_{len(instance.items)}_{seed}.txt', *instance)

In [13]:
problems = []
for gen_type in gen_types:
    for n in [10, 20, 50, 100]:
        for seed in [0]:
            path = f'data/random/{gen_type}_{n}_{seed}.txt'
            problems.append((gen_type, path, Instance(*load_instance(path))))

In [54]:
problems = []
for gen_type in ['random']:
    for n in [10, 20, 50, 100]:
        for seed in [1]:
            path = f'data/random/{gen_type}_{n}_{seed}.txt'
            problems.append((gen_type, path, Instance(*load_instance(path))))

In [18]:
gens, files, instances = zip(*problems)
dataframe = pd.DataFrame(zip(gens, files), columns=['Generator type', 'File'])

In [3]:
time_dataframe = pd.DataFrame(columns=['File', 'Solver', 'Skyline', 'Skyline sorted', 'Concat', 'Annealing skyline'])

In [67]:
for gen, file, inst in problems:
    alg_count = len(dataframe.columns) - 2
    new_row = pd.Series({'Generator type': gen, 'File': file})
    dataframe = dataframe.append(new_row, ignore_index=True)

In [3]:
dataframe = pd.read_csv('data/results.csv')
# noinspection PyRedeclaration
instances = [Instance(*load_instance(file)) for file in dataframe['File']]
time_dataframe = pd.read_csv('data/times.csv')

In [17]:
dataframe.to_csv('data/results.csv', index=False)
time_dataframe.to_csv('data/times.csv', index=False)

In [102]:
algorithms = ['Skyline', 'Skyline sorted', 'Concat', 'Annealing skyline']
df_values = ['Solver solution', 'Solver bound'] + algorithms

In [15]:
# dataframe['Solver solution']=[pd.NA]*len(dataframe)
# dataframe['Solver bound']=[pd.NA]*len(dataframe)
# dataframe['Skyline']=[pd.NA]*len(dataframe)
# dataframe['Skyline sorted']=[pd.NA]*len(dataframe)
# dataframe['Concat']=[pd.NA]*len(dataframe)
# dataframe['Annealing skyline']=[pd.NA]*len(dataframe)
# dataframe['Annealing o-tree']=[pd.NA]*len(dataframe)
# dataframe['NFD']=[pd.NA]*len(dataframe)

In [85]:
for i in range(len(dataframe)):
    if pd.isnull(dataframe['Solver bound'][i]):
        inst = instances[i]
        print('Computing', i)
        start_time = timeit.default_timer()
        _, obj, bound, _ = task_one_model(inst.pallet_width, inst.pallet_height, inst.items, limit=5 * 60 * 1000,
                                          backend='Gurobi')
        end_time = timeit.default_timer()
        dataframe.at[i, 'Solver solution'] = obj
        dataframe.at[i, 'Solver bound'] = bound
        time_dataframe.at[i, 'Solver'] = end_time - start_time

Computing 16
Computing 17
Computing 18
Computing 19


In [80]:
for i in range(len(dataframe)):
    if pd.isnull(dataframe['Skyline'][i]):
        inst = instances[i]
        start_time = timeit.default_timer()
        result, _ = skyline_decode(*inst)
        end_time = timeit.default_timer()
        dataframe.at[i, 'Skyline'] = result
        time_dataframe.at[i, 'Skyline'] = end_time - start_time

In [82]:
for i in range(len(dataframe)):
    if pd.isnull(dataframe['Skyline sorted'][i]):
        inst = instances[i]
        items = inst.items
        start_time = timeit.default_timer()
        order = list(range(len(items)))
        order.sort(key=lambda j: items[j].height * items[j].width, reverse=True)
        result, _ = skyline_decode(*inst, order)
        end_time = timeit.default_timer()
        dataframe.at[i, 'Skyline sorted'] = result
        time_dataframe.at[i, 'Skyline sorted'] = end_time - start_time

In [83]:
for i in range(len(dataframe)):
    if pd.isnull(dataframe['Concat'][i]):
        inst = instances[i]
        start_time = timeit.default_timer()
        result, _ = concat_placement(*inst)
        end_time = timeit.default_timer()
        dataframe.at[i, 'Concat'] = result
        time_dataframe.at[i, 'Concat'] = end_time - start_time

In [17]:
lib = Library(LIBRARY_FILE)
annealing = lib.simulated_annealing

In [ ]:
for i in range(len(dataframe)):
    if pd.isnull(dataframe['Annealing skyline'][i]):
        inst = instances[i]
        print('Computing', i)
        start_time = timeit.default_timer()
        start_temp = max(max(it.width * it.height for it in inst.items), inst.pallet_width * inst.pallet_height / 4)
        result, _ = annealing(*inst, steps=10 ** 6, start_temperature=start_temp)
        end_time = timeit.default_timer()
        dataframe.at[i, 'Annealing skyline'] = result
        time_dataframe.at[i, 'Annealing skyline'] = end_time - start_time

In [12]:
for i in range(len(dataframe)):
    if pd.isnull(dataframe['NFD'][i]):
        inst = instances[i]
        start_time = timeit.default_timer()
        result, _ = NFD_SA.nfd(*inst)
        end_time = timeit.default_timer()
        dataframe.at[i, 'NFD'] = result
        time_dataframe.at[i, 'NFD'] = end_time - start_time

Computing 0
Computing 1
Computing 2
Computing 3
Computing 4
Computing 5
Computing 6
Computing 7
Computing 8
Computing 9
Computing 10
Computing 11
Computing 12
Computing 13
Computing 14
Computing 15
Computing 16
Computing 17
Computing 18
Computing 19


In [ ]:
for i in range(len(dataframe)):
    if pd.isnull(dataframe['Annealing o-tree'][i]):
        inst = instances[i]
        print('Computing', i)
        start_time = timeit.default_timer()
        result, _ = NFD_SA.simulated_annealing(*inst, cooling_coef=0.99, num_of_iters=500000)
        end_time = timeit.default_timer()
        dataframe.at[i, 'Annealing o-tree'] = result
        time_dataframe.at[i, 'Annealing o-tree'] = end_time - start_time


In [13]:
dataframe

,Generator type,File,Solver solution,Solver bound,Skyline,Skyline sorted,Concat,Annealing skyline,NFD,Annealing o-tree
0,random,data/random/random_10_0.txt,8288.0,8288.0,8288,8288,8568,8288,8288,8594
1,random,data/random/random_20_0.txt,6540.0,6540.0,6540,6540,6540,6540,6540,6540
2,random,data/random/random_50_0.txt,2181.0,2181.0,2181,2181,2181,2181,2181,3961
3,random,data/random/random_100_0.txt,1734.0,-5285.0,93,29,88,0,983,3333
4,guillotine_rand,data/random/guillotine_rand_10_0.txt,0.0,0.0,0,0,0,0,4120,2961
5,guillotine_rand,data/random/guillotine_rand_20_0.txt,368.0,0.0,2258,775,187,143,3697,3687
6,guillotine_rand,data/random/guillotine_rand_50_0.txt,897.0,0.0,490,286,1026,35,2658,4988
7,guillotine_rand,data/random/guillotine_rand_100_0.txt,1739.0,0.0,364,431,648,0,2043,4035
8,guillotine_equal,data/random/guillotine_equal_10_0.txt,0.0,0.0,0,0,0,0,3337,2357
9,guillotine_equal,data/random/guillotine_equal_20_0.txt,462.0,0.0,1232,958,1646,0,3860,4766


In [11]:
def form(x, area):
    if isinstance(x, int) or isinstance(x, float):
        return f'{1 - x / area:.2%}'
    else:
        return x


def form2(x, area):
    if isinstance(x, int) or isinstance(x, float):
        return f'{100 * (1 - x / area):.2f}'
    else:
        return x


def form3(x):
    if isinstance(x, float):
        return round(x, 4)
    else:
        return x

In [7]:
dataframe_rel = dataframe.apply(
        lambda r, ind: r.apply(form, args=(ind[r.name].pallet_width * ind[r.name].pallet_height,)),
        args=(instances,), axis=1)
dataframe_rel

,Generator type,File,Solver solution,Solver bound,Skyline,Skyline sorted,Concat,Annealing skyline,NFD,Annealing o-tree
0,random,data/random/random_10_0.txt,17.12%,17.12%,17.12%,17.12%,14.32%,17.12%,17.12%,14.06%
1,random,data/random/random_20_0.txt,34.60%,34.60%,34.60%,34.60%,34.60%,34.60%,34.60%,34.60%
2,random,data/random/random_50_0.txt,78.19%,78.19%,78.19%,78.19%,78.19%,78.19%,78.19%,60.39%
3,random,data/random/random_100_0.txt,82.66%,152.85%,99.07%,99.71%,99.12%,100.00%,90.17%,66.67%
4,guillotine_rand,data/random/guillotine_rand_10_0.txt,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,58.80%,70.39%
5,guillotine_rand,data/random/guillotine_rand_20_0.txt,96.32%,100.00%,77.42%,92.25%,98.13%,98.57%,63.03%,63.13%
6,guillotine_rand,data/random/guillotine_rand_50_0.txt,91.03%,100.00%,95.10%,97.14%,89.74%,99.65%,73.42%,50.12%
7,guillotine_rand,data/random/guillotine_rand_100_0.txt,82.61%,100.00%,96.36%,95.69%,93.52%,100.00%,79.57%,59.65%
8,guillotine_equal,data/random/guillotine_equal_10_0.txt,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,66.63%,76.43%
9,guillotine_equal,data/random/guillotine_equal_20_0.txt,95.38%,100.00%,87.68%,90.42%,83.54%,100.00%,61.40%,52.34%


In [8]:
dataframe_rel.to_csv('data/results_relative.csv', index=False)

In [16]:
time_dataframe

,File,Solver,Skyline,Skyline sorted,Concat,Annealing skyline,NFD,Annealing o-tree
0,data/random/random_10_0.txt,1.624492,0.000327,0.000338,0.002474,3.602594,2.034517,2.277815
1,data/random/random_20_0.txt,0.501135,0.000794,0.000797,0.006357,7.543365,1.840237,5.549176
2,data/random/random_50_0.txt,1.227332,0.003830,0.003682,0.007344,16.997357,0.189086,60.720009
3,data/random/random_100_0.txt,300.000766,0.011746,0.009052,0.467700,40.091562,0.095007,134.436092
4,data/random/guillotine_rand_10_0.txt,1.014615,0.000508,0.000434,0.001261,2.026347,0.106238,0.218222
5,data/random/guillotine_rand_20_0.txt,300.007164,0.001496,0.001980,0.006144,5.156990,0.104167,2.685331
6,data/random/guillotine_rand_50_0.txt,300.004832,0.007298,0.008502,0.021831,17.154622,0.100824,48.207526
7,data/random/guillotine_rand_100_0.txt,300.004410,0.034950,0.030196,0.178921,46.501068,0.105599,128.938798
8,data/random/guillotine_equal_10_0.txt,1.399953,0.000402,0.000351,0.001008,1.908792,0.067391,0.140566
9,data/random/guillotine_equal_20_0.txt,300.004599,0.001196,0.001142,0.010154,5.094101,0.092094,2.893904


In [38]:
dataframe_rel2 = dataframe.apply(
        lambda r, ind: r.apply(form2, args=(ind[r.name].pallet_width * ind[r.name].pallet_height,)),
        args=(instances,), axis=1)
time_form = time_dataframe.applymap(form3)
# df=dataframe_rel2.merge(time_dataframe,on='File')
algorithms = ['Solver', 'Skyline', 'Skyline sorted', 'Concat', 'Annealing skyline', 'NFD', 'Annealing o-tree']
flt2 = pd.Series([not re.search('random_\d*_0\.txt', dataframe_rel2['File'][i]) for i in range(len(dataframe_rel2))])
for item_count in [10, 20, 50, 100]:
    flt = pd.Series([len(inst.items) == item_count for inst in instances])
    # noinspection PyRedeclaration
    df = dataframe_rel2.loc[flt].loc[flt2].rename(columns={'Solver solution': 'Solver'}).filter(
            items=algorithms).reset_index()
    tdf = time_form.loc[flt].loc[flt2].filter(items=algorithms).reset_index()
    res_df = pd.DataFrame()
    for i in range(len(df)):
        for alg in algorithms:
            res_df.at[2 * i, alg] = df[alg][i]
            res_df.at[2 * i + 1, alg] = tdf[alg][i]
    res_df.T.to_csv(f'data/result_{item_count}.csv', header=False)

In [8]:
dataframe_mass = pd.DataFrame({'Generator type': dataframe['Generator type']})

for i in range(len(dataframe)):
    w, h, its = instances[i]
    its2 = [Item(itm.width, itm.height, random.randint(1, 5)) for itm in its]
    file = dataframe['File'][i].replace('data/random', 'data/random_mass')
    save_instance(file, w, h, its2)
    dataframe_mass.at[i, 'File'] = file

In [12]:
# times_mass=pd.DataFrame({'File':dataframe_mass['File']})

In [3]:
dataframe_mass = pd.read_csv('data/results_mass.csv')
instances_mass = [Instance(*load_instance(file)) for file in dataframe_mass['File']]
times_mass = pd.read_csv('data/times_mass.csv')

In [6]:
dataframe_mass.to_csv('data/results_mass.csv', index=False)
times_mass.to_csv('data/times_mass.csv', index=False)

In [11]:
# dataframe_mass['Solver solution']=[pd.NA]*len(dataframe)
# dataframe_mass['Solver bound']=[pd.NA]*len(dataframe)

In [5]:
for i in range(len(dataframe_mass)):
    if pd.isnull(dataframe_mass['Solver bound'][i]):
        inst = instances_mass[i]
        print('Computing', i)
        start_time = timeit.default_timer()
        _, obj, bound, _ = task_two_model(inst.pallet_width, inst.pallet_height, inst.items, 0.2 * inst.pallet_width,
                                          0.2 * inst.pallet_height, limit=5 * 60 * 1000, backend='Gurobi')
        end_time = timeit.default_timer()
        dataframe_mass.at[i, 'Solver solution'] = obj
        dataframe_mass.at[i, 'Solver bound'] = bound
        times_mass.at[i, 'Solver'] = end_time - start_time

Computing 0
Computing 1
Computing 2
Computing 3
Computing 4
Computing 5
Computing 6
Computing 7
Computing 8
Computing 9
Computing 10
Computing 11
Computing 12
Computing 13
Computing 14
Computing 15
Computing 16
Computing 17
Computing 18
Computing 19


In [15]:
mass_df = dataframe_mass.apply(
    lambda r, ind: r.apply(form2, args=(ind[r.name].pallet_width * ind[r.name].pallet_height,)), args=(instances_mass,),
    axis=1).merge(times_mass.applymap(form3), on='File').rename(
    columns={'Solver': 'Time', 'Solver solution': 'Solution', 'Solver bound': 'Bound'}).drop('File', axis=1).applymap(
    lambda x: x.replace('_', ' ') if isinstance(x, str) else x)
mass_df.to_csv('data/mass_formatted.csv', index=False)